## ANN Implementation

In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import regularizers
import datetime
import pickle

# Reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# ----------------- Load and clean dataset -----------------
data = pd.read_excel("Churn_Modelling_sample.csv.xlsx")
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# Drop irrelevant columns
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

# Encode Gender
if "Gender" in data.columns:
    label_encoder_gender = LabelEncoder()
    data["Gender_encoded"] = label_encoder_gender.fit_transform(data["Gender"])
    data = data.drop("Gender", axis=1)
    with open("label_encoder_gender.pkl", "wb") as file:
        pickle.dump(label_encoder_gender, file)

# OneHot encode Geography (scikit-learn ≥ 1.2 syntax)
if "Geography" in data.columns:
    onehot_encoder_geo = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    geo_encoded = onehot_encoder_geo.fit_transform(data[["Geography"]])
    geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))
    data = pd.concat([data.drop("Geography", axis=1), geo_encoded_df], axis=1)
    with open("onehot_encoder_geo.pkl", "wb") as file:
        pickle.dump(onehot_encoder_geo, file)

# ----------------- Features & Target -----------------
X = data.drop("Exited", axis=1)
y = data["Exited"]
X = X.select_dtypes(include=[np.number])  # Ensure numeric only

# ----------------- Handle rare classes before stratify -----------------
class_counts = y.value_counts()
rare_classes = class_counts[class_counts < 2].index
if len(rare_classes) > 0:
    print(f"Removing rare classes: {list(rare_classes)}")
    mask = ~y.isin(rare_classes)
    X, y = X[mask], y[mask]
    class_counts = y.value_counts()

# Dynamically adjust test_size if needed
min_class_count = class_counts.min()
safe_test_size = min(0.1, (min_class_count - 1) / min_class_count)
if safe_test_size <= 0:
    safe_test_size = 0.1  # fallback

# ----------------- Train/Test Split -----------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=safe_test_size,
    random_state=42,
    stratify=y
)

# ----------------- Scaling -----------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

# ----------------- Class Weights -----------------
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(zip(np.unique(y_train), class_weights))

# ----------------- Build Model -----------------
model = Sequential([
    Dense(128, activation="relu", kernel_regularizer=regularizers.l2(0.001), input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.4),

    Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.2),

    Dense(1, activation="sigmoid")
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# ----------------- Callbacks -----------------
log_dir = "logs/fit_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=8, min_delta=0.0005, restore_best_weights=True, verbose=1)
reduce_lr_callback = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=4, min_lr=1e-6, verbose=1)

# ----------------- Train -----------------
history = model.fit(
    X_train_scaled,
    y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[tensorboard_callback, early_stopping_callback, reduce_lr_callback],
    class_weight=class_weights,
    verbose=1
)

# ----------------- Save Model -----------------
model.save('churn_model.h5')

# ----------------- Evaluation -----------------
y_pred_prob = model.predict(X_test_scaled).ravel()
y_pred = (y_pred_prob > 0.5).astype(int)

print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_prob))


Removing rare classes: [0]


c:\Users\Asus\ANN Classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (None, 128)            │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,801 (50.00 KB)

...

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.3333 - loss: 1.1051 - val_accuracy: 0.0000e+00 - val_loss: 0.9906 - learning_rate: 0.0010
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step - accuracy: 0.6667 - loss: 0.8938 - val_accuracy: 0.0000e+00 - val_loss: 1.0064 - learning_rate: 0.0010
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - accuracy: 0.6667 - loss: 1.5111 - val_accuracy: 0.0000e+00 - val_loss: 1.0044 - learning_rate: 0.0010
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.3333 - loss: 1.0926 - val_accuracy: 0.0000e+00 - val_loss: 1.0063 - learning_rate: 0.0010
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.3333 - loss: 1.1340 - val_accuracy: 0.0000e+00 - val_loss: 0.9867 - learning_rate: 0.0010
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - accuracy: 0.3333 - loss: 0.8216 - val_accuracy: 0.0000e+00 - val_loss: 0.9633 - learning_rate: 0.0010
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step - accuracy: 0.3333 - l

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step


c:\Users\Asus\ANN Classification\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Asus\ANN Classification\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Asus\ANN Classification\venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0


Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

Confusion Matrix:
 [[0 0]
 [1 0]]
ROC-AUC Score: nan


c:\Users\Asus\ANN Classification\venv\Lib\site-packages\sklearn\metrics\_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [32]:
X_train_scaled[:,0].mean()

np.float64(0.0)

In [33]:
X_train_scaled[:,0].std()

np.float64(0.9999999999999999)

In [39]:
%load_ext tensorboard
%tensorboard --logdir logs/fit_20250815-124656

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6012 (pid 12376), started 0:00:03 ago. (Use '!kill 12376' to kill it.)

In [15]:
X_train_scaled.shape

(4, 11)

In [16]:
X_train.shape

(4, 11)